<img src="../../shared/img/banner.svg"></img>

# Extra Slides - `pycoin`

In [ ]:
%matplotlib notebook

In [ ]:
import sys

sys.path.append("../../")

from shared.src import quiet
from shared.src import seed

In [ ]:
import math
import random
import string
import time

import daft
from IPython.display import Image
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import pymc3 as pm
import scipy.special
import scipy.stats
import seaborn as sns

In [ ]:
def sample_from(model, n, filt=10):
    with model:
        samples = pm.sample(chains=1, draws=n * filt)[::filt]
    return samples

In [ ]:
def samples_to_dataframe(samples):
    return pd.DataFrame([sample for sample in samples])

def add_counts(data):
    data["count"] = np.ones(len(data))
    return data

## pycoin

Speaking loosely, cryptocurrencies work as follows:
a transaction, such as `"alice paid bob 10 coins"`,
is considered verified if the result of applying a function called `hash` to it
results in a number smaller than some value.
`hash` can be any function that takes in a string and returns a number.
In order to get `hash` to evaluate to a small number,
you're allowed to append whatever gibberish you want to the transaction.

Some folks collect transactions and attempt to verify them,
which they do by searching for a gibberish string to add to the transaction.
This string is called a "nonce", or "Number used ONCE".

Why would they do that?
In exchange, they are allowed to add a special transaction before they go searching:
`"{somebody} gets 1 coin"`.
Normally, they choose themselves.

Since it produces currency, this process is called "mining",
by analogy with mining precious metal currency.
Since there are usually many transactions combined together into a chunk,
the whole process is called "mining a block".

In [ ]:
data = "alice paid bob 10 pycoin; {name} gets 1 pycoin. nonce:".format(name="charles")

def run_pycoin(data, target=1e16):
    s = data 

    while not target > hash(s) > -target:
        s += random.choice(string.ascii_letters)
        
    return s

In [ ]:
run_pycoin(data, target=1e17)

The function `hash` is usually chosen to be _cryptographic_, one of the reasons for the _crypto_ in _cryptocurrency_.
One thing that this means is that its behavior is believed to be extremely unpredictable:
the only way to know the output of `hash` on some value is to run it on that value;
the only way to find an input that makes `hash` give a certain output is to try it on lots of inputs.
There is no simple pattern that would allow, say, someone very clever to find a gibberish string much faster and with much less effort than everyone else.

The strategy almost every miner takes is, then, to try _random inputs_.
This means that we can model cryptocurrency mining with our random variable toolkit.

Even better: putting a random input into `hash` and checking whether the output is below a certain number is a kind of random variable we're very familiar with.
The chance that any given input succeeds is very low, but we will be trying a large number of them.
Or, from another perspective, each attempt is completely independent of the previous, and so the mining process is memoryless.

We can therefore use the same model we used for the raindrops as our model of `pycoin` mining!

The time in between successfully mined blocks is `Exponential`; the number of blocks mined in any given period (second, day, month) is `Poisson`.

### HISTORICAL BLOCKCHAIN DATA HERE

In [ ]:
def test_pycoin(data, num_blocks=1000, target=1e16):
    times = []
    for block_ii in range(num_blocks):
        start_time = time.time()
        run_pycoin(data, target)
        elapsed_time = time.time() - start_time
        times.append(elapsed_time)
        
    return times

In [ ]:
times = test_pycoin(data, num_blocks=5000, target=1e18)

In [ ]:
plt.figure()
plt.hist(times, normed=True, bins=500);

In [ ]:
beta = np.mean(times)

In [ ]:
def exp_pdf(t, beta):
    return 1 / beta * np.exp(- t / beta)

In [ ]:
ts = np.linspace(0, max(times))
plt.plot(ts, exp_pdf(ts, beta), color="C1", lw=4);